In [24]:
# -*- coding:utf-8 -*-
import numpy as np
import pandas as pd
import pickle
import itertools
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import random
import csv
import math
import xlsxwriter
from io import BytesIO

############# Data Processing, Network Construction #############

def find_neighbors(adj_matrix, node):
    neighbors = []
    for f_i, connected in enumerate(adj_matrix[node]):
        if connected == 1:
            neighbors.append(f_i)
    return neighbors


def find_rank_descending(number, num_list):
    num_list.append(number)
    sorted_list = sorted(num_list, reverse=True)
    rank = sorted_list.index(number) + 1
    return rank


def csv_to_list(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            return list(row)


def csv_column_to_list(file_path):
    data_list = []
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            data_list.append(row[0])
    return data_list


def remove_elements(list1, list2):
    return [element for element in list1 if element not in list2]


# File Reading
mol_interact_relations = pickle.load(open('./Data/mol_interact_relations.pkl', 'rb'))
mol_index = pickle.load(open('./Data/mol_index.pkl', 'rb'))
mol_syno = pickle.load(open('./Data/mol_syno.pkl', 'rb'))
can_mol_index = pickle.load(open('./Data/can_mol_index.pkl', 'rb'))
mol_fps = pickle.load(open('Data/KR/KR.pkl', 'rb'))

get_num = 200  # Number of Predicted CCFs
input_smi = "C1CCC(C1)(C#N)C2=CC=C(C=C2)NC(=O)C3=C(N=CC=C3)NCC4=CC=NC=C4" # Molecule to be Predicted
num_molecule = len(mol_index)
substr_weight = 1   # Initial Weights

can_mol_dict = dict(can_mol_index)
input_mol = Chem.MolFromSmiles(input_smi)
can_input_smi = Chem.MolToSmiles(input_mol, isomericSmiles=False)
input_smi_index = can_mol_dict.get(can_input_smi, len(can_mol_index))
print('Target molecule SMILES and index', can_input_smi, input_smi_index)

# If it is an external molecule, the target_m.csv fingerprint file will be called here.
mol_input_fps = mol_fps
if input_smi_index >= len(mol_index):
    input_mol = Chem.MolFromSmiles(can_input_smi)
    input_fp = csv_to_list('./Data/target_m.csv')
    mol_input_fps.append(input_fp)
    print("External molecules, please ensure that the target_m.csv file is the fingerprint file for the target molecules.")
    
substructure_matrix = np.array(mol_input_fps, dtype=np.float64)
substructure_matrix = substructure_matrix[:, np.sum(substructure_matrix, axis=0) != 0]
mol_num, substructure_num = substructure_matrix.shape
substructure_links = []
for mol in range(mol_num):
    for i in range(substructure_num):
        if substructure_matrix[mol, i] == 1:
            substructure_links.append([mol, mol_num + i])

substructure_links = [item + [substr_weight] for item in substructure_links]
mol_interact_relations = [item + [1/substr_weight] for item in mol_interact_relations]
links = mol_interact_relations + substructure_links

mat_nodes = list(itertools.chain.from_iterable(links))
mat_nodes = set(mat_nodes)
mat_nodes = {np.int32(node) for node in mat_nodes}
mat_size = np.int32(max(mat_nodes) + 1)

network = np.zeros((mat_size, mat_size))
for item in links:
    network[np.int32(item[0]), np.int32(item[1])] = item[2]
network = network + network.T

count_ones_per_row = np.sum(network == 1, axis=1)
count_ones_per_row_list = count_ones_per_row.tolist()
degree_list_all = [x + 1 for x in count_ones_per_row_list]  # 0 cannot be a divisor, so add 1 to all

############# Hyperparameter Selection #############

NBI_Model = 'KR'
if NBI_Model == 'KR':
    Alpha = 0.3
    Beta = 0.4
    Gama = 0.5
elif NBI_Model == 'FCFP4':
    Alpha = 0.4
    Beta = 0.4
    Gama = 0.6
elif NBI_Model == 'ECFP4':
    Alpha = 0.3
    Beta = 0.4
    Gama = 0.6

Emm = 1 - Beta
Esm = Beta

############# Execute the NBI algorithm #############

adj_matrix = network.copy()
target = input_smi_index
num_nodes = len(adj_matrix)
scores = [0] * num_nodes  # Initial Score List

# Assign different values to molecules and substructures based on Alpha
neighbors_1 = find_neighbors(adj_matrix, target)
s_neighbors_of_target_node = [num for num in neighbors_1 if num > num_molecule]
for i_n in neighbors_1:
    scores[i_n] = (1 - Alpha)
for i_s in s_neighbors_of_target_node:
    scores[i_s] = Alpha

############# k=1，Perform the first resource propagation
nodes_value1 = []
# Traverse all nodes and collect the indices of nodes with resources into the list nodes_value1
for i_2 in range(len(scores)):
    if scores[i_2] != 0:
        nodes_value1.append(i_2)
# Traverse all nodes with resources, distribute all their resources evenly among their neighbors, and update the scores list
for k1 in nodes_value1:
    neighbors_k1 = find_neighbors(adj_matrix, k1)
    score_k1 = scores[k1]
    if k1 >= num_molecule:
        give_score1 = score_k1 / len(neighbors_k1)
        scores[k1] = 0
        for n1 in neighbors_k1:
            scores[n1] += give_score1
    else:
        Neigh_S1 = 0
        for count_NS1 in neighbors_k1:
            if count_NS1 < num_molecule:
                Neigh_S1 += 1

        Neigh_M1 = len(neighbors_k1) - Neigh_S1
        # Allocate resources to molecules and substructures based on beta
        attend_M1 = score_k1 * (Emm / (Emm * Neigh_M1 + Esm * Neigh_S1))
        attend_S1 = score_k1 * (Esm / (Emm * Neigh_M1 + Esm * Neigh_S1))
        scores[k1] = 0
        for n1 in neighbors_k1:
            if n1 < num_molecule:
                scores[n1] += attend_M1
            else:
                scores[n1] += attend_S1

############# K=2，Perform the second resource propagation
new_hub2 = []
for i_3 in range(len(scores)):
    if scores[i_3] != 0:
        new_hub2.append(i_3)
for k2 in new_hub2:
    neighbors_k2 = find_neighbors(adj_matrix, k2)
    score_k2 = scores[k2]
    if k2 >= num_molecule:
        give_score2 = score_k2 / len(neighbors_k2)
        scores[k2] = 0
        for n2 in neighbors_k2:
            scores[n2] += give_score2

    else:
        Neigh_S2 = 0
        for count_NS2 in neighbors_k2:
            if count_NS2 < num_molecule:
                Neigh_S2 += 1

        Neigh_M2 = len(neighbors_k2) - Neigh_S2
        attend_M2 = score_k2 * (Emm / (Emm * Neigh_M2 + Esm * Neigh_S2))
        attend_S2 = score_k2 * (Esm / (Emm * Neigh_M2 + Esm * Neigh_S2))
        scores[k2] = 0
        for n2 in neighbors_k2:
            if n2 < num_molecule:
                scores[n2] += attend_M2
            else:
                scores[n2] += attend_S2

############# Impose penalties based on gamma #############

degree_list = count_ones_per_row_list.copy()
degree_list = degree_list[:num_molecule]  # Only calculate the molecular part, discard the substructure segments
degree_list = [x ** Gama for x in degree_list]
scores = scores[:num_molecule]
scores = [s_elem / y_elem for s_elem, y_elem in zip(scores, degree_list)]

# Convert to indices for sorting
sorted_indices = sorted(range(len(scores)), key=lambda x: scores[x], reverse=True)

# Remove neighboring nodes, remove substructure nodes.
result_list = [item for item in sorted_indices if item not in neighbors_1]
reversed_dict = {v: k for k, v in can_mol_index.items()}
smiles_list = []
for i in result_list:
    if i in reversed_dict:
        smiles_i = reversed_dict[i]
        smiles_list.append(smiles_i)

smiles_list = smiles_list[:get_num]
file_path = './Data/hub_nodes.csv'
data_list = csv_column_to_list(file_path)
result = remove_elements(smiles_list, data_list)

############# Result Processing #############

csv_filename = f"./Results/results-{NBI_Model}.csv"
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Synonym', 'SMILES', 'Rank'])
    i = 1
    for smiles in result:
        writer.writerow(['molecule',  smiles, i])
        i += 1

# Identify the existing CCFs of the target molecule in the network
target_neighbors_list = []
for index in neighbors_1:
    smiles = next((item[0] for item in mol_index if item[1] == index), None)
    if smiles:
        target_neighbors_list.append(smiles)
print('Existing CCFs of the target molecule：', target_neighbors_list)

common_items_indices = [result.index(item) for item in target_neighbors_list if item in result]
common_items_indices = [index + 1 for index in common_items_indices]

# Set the Rank value of known molecules to 'Known'
common_items_indices = []
csv_filename = f"./Results/results-{NBI_Model}.csv"
with open(csv_filename, 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    rows = list(reader)

for index in common_items_indices:
    if index < len(rows) and rows[index]:
        rows[index] = rows[index][:-1] + ['Known']
        
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(rows)

############# Result Output #############

file_name = f'./Results/results-{NBI_Model}'

def mol_to_excel(file):
    header = ['Synonym', 'SMILES', 'Rank']
    item_style = {
        'align': 'center',
        'valign': 'vcenter',
        'top': 2,
        'left': 2,
        'right': 2,
        'bottom': 2,
        'text_wrap': 1
    }
    header_style = {
        'bold': 1,
        'valign': 'vcenter',
        'align': 'center',
        'top': 2,
        'left': 2,
        'right': 2,
        'bottom': 2
    }
    workbook = xlsxwriter.Workbook(f'{file}.xlsx')
    ItemStyle = workbook.add_format(item_style)
    HeaderStyle = workbook.add_format(header_style)
    worksheet = workbook.add_worksheet()
    worksheet.set_column('A:A', 38)
    worksheet.set_column('B:B', 40)
    worksheet.set_column('C:C', 10)
    for ix_, i in enumerate(header):
        worksheet.write(0, ix_, i, HeaderStyle)

    df = pd.read_csv(f'{file}.csv')

    for i in range(df.shape[0]):
        synonym = df.iloc[i, 0]
        structure_smi = df.iloc[i, 1]
        rank = df.iloc[i, 2]
        img_data_structure = BytesIO()
        c_structure = Chem.MolFromSmiles(structure_smi)
        img_structure = Draw.MolToImage(c_structure)
        img_structure.save(img_data_structure, format='PNG')

        worksheet.set_row(i + 1, 185)
        worksheet.insert_image(i + 1, 0, 'f',
                               {'x_scale': 0.9, 'y_scale': 0.8, 'image_data': img_data_structure, 'positioning': 1})
        worksheet.write(i + 1, 1, structure_smi, ItemStyle)
        worksheet.write(i + 1, 2, rank, ItemStyle)

    workbook.close()

mol_to_excel(file_name)
print("Over, Generate a new Excel file")

Target molecule SMILES and index N#CC1(c2ccc(NC(=O)c3cccnc3NCc3ccncc3)cc2)CCCC1 714
Existing CCFs of the target molecule： ['O=C(O)CCCCC(=O)O', 'O=C(O)C(O)c1ccccc1', 'O=C(O)CCCCCCCCC(=O)O']
Generate a new Excel file


In [14]:
# Identify the existing CCFs of the target molecule in the network
target_neighbors_list = []
for index in neighbors_1:
    smiles = next((item[0] for item in mol_index if item[1] == index), None)
    if smiles:
        target_neighbors_list.append(smiles)
print('Existing CCFs of the target molecule：', target_neighbors_list)

common_items_indices = [result.index(item) for item in target_neighbors_list if item in result]
common_items_indices = [index + 1 for index in common_items_indices]
print(common_items_indices)

# Set the Rank value of known molecules to 'Known'
common_items_indices = []
csv_filename = f"./Results/results-{NBI_Model}.csv"
with open(csv_filename, 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    rows = list(reader)

for index in common_items_indices:
    if index < len(rows) and rows[index]:
        rows[index] = rows[index][:-1] + ['Known']
        
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(rows)

['O=C(O)CCCCC(=O)O', 'O=C(O)C(O)c1ccccc1', 'O=C(O)CCCCCCCCC(=O)O']
